In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

# Set the directory where the CSV files are located, usually it's '/content' in Colab
file_directory = '/content/drive/MyDrive/CS411_PT1/Dataset/Airbnbs'

# List all CSV files in the directory
csv_files = [file for file in os.listdir(file_directory) if file.endswith('.csv')]


In [ ]:
# Define columns to delete
columns_to_delete = ['scrape_id', 'last_scraped','source','host_since','host_location','host_about','host_thumbnail_url','host_picture_url','host_neighbourhood','host_listings_count','host_total_listings_count','host_verifications','host_has_profile_pic','neighbourhood','neighbourhood_cleansed',
'neighbourhood_group_cleansed','bathrooms_text','minimum_nights','maximum_nights','minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm','calendar_updated','has_availability','availability_30','availability_60',
'availability_90','availability_365','calendar_last_scraped','number_of_reviews_ltm','number_of_reviews_l30d','first_review','last_review','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','license','instant_bookable',
'calculated_host_listings_count','host_response_time','room_type','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms','reviews_per_month']

# Create a list to store the modified DataFrames
modified_dfs = []

for file in csv_files:
    # Load each CSV file
    df = pd.read_csv(os.path.join(file_directory, file))

    # Delete specified columns
    df.drop(columns=columns_to_delete, inplace=True, errors='ignore')

    # Add the 'close_to_airport' column with the file name (without '.csv')
    df['close_to_airport'] = os.path.splitext(file)[0]

    # Append modified DataFrame to the list
    modified_dfs.append(df)


<ipython-input-16-01becfbbe39d>:12: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(file_directory, file))


In [ ]:
merged_df = pd.concat(modified_dfs, ignore_index=True)

In [ ]:
merged_df.to_csv('/content/drive/MyDrive/CS411_PT1/Dataset/merged_file2.csv', index=False)

### Cleaning the values


In [ ]:
# import pandas as pd
# import json

# df = pd.read_csv('/content/drive/MyDrive/CS411_PT1/Dataset/merged_file2.csv')
# df = df.copy()

# # Convert boolean columns
# boolean_columns = ['host_is_superhost', 'host_identity_verified']
# for col in boolean_columns:
#     df[col] = df[col].map({'t': True, 'f': False})

# # Clean price column
# df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)

# # Convert percentage strings to decimals
# df['host_response_rate'] = df['host_response_rate'].str.rstrip('%').astype(float)
# df['host_acceptance_rate'] = df['host_acceptance_rate'].str.rstrip('%').astype(float)

# # Convert amenities string to JSON
# df['amenities'] = df['amenities'].apply(json.dumps)

# # # Convert review score to decimal
# # df['review_scores_rating'] = df['review_scores_rating'] / 5  # Assuming 5 is max score
# df.to_csv('/content/drive/MyDrive/CS411_PT1/merged_file3.csv', index=False)



In [ ]:
import re
import pandas as pd
import json

def is_valid_text(text):
    """
    Check if text contains only allowed characters:
    - alphanumeric
    - space
    - !, ., ,, -, ;, :
    """
    if pd.isna(text):  # Handle None/NaN values
        return False

    # Create regex pattern for allowed characters
    pattern = r'^[a-zA-Z0-9\s!,.\-;:\'\"#@$%&*()_+=\[\]{}/<>?\\|`~]*$'
    return bool(re.match(pattern, text))


df = pd.read_csv('/content/drive/MyDrive/CS411_PT1/Dataset/merged_file2.csv')
df = df.copy()

# Create mask for valid text fields
text_mask = (
    df['name'].apply(is_valid_text) &
    df['description'].apply(is_valid_text) &
    df['neighborhood_overview'].apply(is_valid_text)
)

# Filter out records with invalid text
print(f"Records before text validation: {len(df)}")
df = df[text_mask]
print(f"Records after text validation: {len(df)}")
print(f"Removed {len(df) - sum(text_mask)} records due to invalid characters")

# Continue with other preprocessing steps
# Convert boolean columns
boolean_columns = ['host_is_superhost', 'host_identity_verified']
for col in boolean_columns:
    df[col] = df[col].map({'t': True, 'f': False})

# Clean price column
df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)

# Convert percentage strings to decimals
df['host_response_rate'] = df['host_response_rate'].str.rstrip('%').astype(float)
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.rstrip('%').astype(float)

# Convert amenities string to JSON
df['amenities'] = df['amenities'].apply(json.dumps)


df.to_csv('/content/drive/MyDrive/CS411_PT1/Dataset/merged_file3.csv', index=False)



Records before text validation: 125943
Records after text validation: 45720
Removed 0 records due to invalid characters


### Data type conversion

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import re

# Load your DataFrame (update with your actual CSV path)
df = pd.read_csv('/content/drive/MyDrive/CS411_PT1/Flights_latest.csv')

# Sample data for demonstration (replace this with your loaded data)
# df = pd.DataFrame({
#     'travel_duration': ['PT2H51M', 'PT1H24M', 'PT6H27M', 'PT3H48M'],
#     'departure_time': ['2022-05-29T07:14:00.000-05:00', '2022-06-10T17:32:00.000-07:00'],
#     'arrival_time': ['2022-05-29T10:05:00.000-05:00', '2022-06-10T20:45:00.000-07:00']
# })

# Function to convert ISO 8601 duration format (e.g., 'PT2H51M') to minutes
def duration_to_minutes(duration):
    match = re.match(r'PT(?:(\d+)H)?(?:(\d+)M)?', duration)
    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    return hours * 60 + minutes

# Apply the function to the 'travel_duration' column
df['travel_duration'] = df['travel_duration'].apply(duration_to_minutes)

# Function to extract only the time part from a datetime string
def extract_time(datetime_str):
    return pd.to_datetime(datetime_str).time()

# Apply the function to 'departure_time' and 'arrival_time'
df['departure_time'] = df['departure_time'].apply(extract_time)
df['arrival_time'] = df['arrival_time'].apply(extract_time)

# Save to a new CSV file
df.to_csv('/content/drive/MyDrive/CS411_PT1/Flights_file.csv', index=False)



In [7]:
import pandas as pd
from datetime import datetime, timedelta
import re

# Load your DataFrame (update with your actual CSV path)
df = pd.read_csv('/content/drive/MyDrive/CS411_PT1/Dataset/Flight.csv')

# Convert 'date_column' to YYYY-MM-DD format
df['flight_date'] = pd.to_datetime(df['flight_date'], format='%d/%m/%y').dt.strftime('%Y-%m-%d')

# Save to a new CSV file
df.to_csv('/content/drive/MyDrive/CS411_PT1/Dataset/modified_file.csv', index=False)
